<a href="https://colab.research.google.com/github/GabrielData21/ETL-com-PySpark/blob/main/ETL_com_Python_e_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de ETL com Python (PySpark)

**Descrição da atividade**

O objetivo desse projeto é desenvolver um processo de ETL utilizando liguagem Python e o framework PySpark, considerando 3 estágios de transformação dos dados: raw, staging e refined. Foram utilizados os dados do ENEM 2020 para o exercício desse projeto.


Usei o chatGPT pra conseguir alguns comandos que vão ajudar na execução do ETL.

## Instalando PySpark

A partir da versão 3.4.0 é possível realizar a instalação do PySpark no Google Colab de forma isolada. Em versões anteriores era necessário instalar o ambiente com Hadoop, java e outros módulos.

In [ ]:
# Instalando o PySpark
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=e39478cb0f4676a0a4e043d9007a608d2c6d4cda06fede9bd82c2de2d44a2048
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
# Iniciando sessão do PySpark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL-enem").getOrCreate()
spark

# Organizando o ambiente para os arquivos do ETL

Solicitei ao chatGPT os comandos adequados para criar as pastas onde vamos guardar os arquivos do ETL em seus 3 estágios: raw, staging e refined. Os comando são os mesmos utilizados no Linux Shell para criação de diretórios.

In [ ]:
# Criando a pasta mestre do projeto
!mkdir enem_2020/

In [ ]:
# Criando as pastas dos estágios do ETL
!mkdir enem_2020/1.raw/ enem_2020/2.staging/ enem_2020/3.refined/

## Aquisição de dados ENEM

### Download e pré processamento dos dados

In [ ]:
# Captação dos dados do ENEM 2020
!wget https://download.inep.gov.br/microdados/microdados_enem_2020.zip --no-check-certificate

--2023-08-31 21:28:52--  https://download.inep.gov.br/microdados/microdados_enem_2020.zip
Resolving download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Connecting to download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 620776982 (592M) [application/zip]
Saving to: ‘microdados_enem_2020.zip’

microdados_enem_202 100%[===================>] 592.02M   413KB/s    in 21m 40s 

2023-08-31 21:50:33 (466 KB/s) - ‘microdados_enem_2020.zip’ saved [620776982/620776982]

total 606240
drwxr-xr-x 4 root root      4096 Aug 31 21:28 enem_2020
-rw-r--r-- 1 root root 620776982 Feb 16  2022 microdados_enem_2020.zip
drwxr-xr-x 1 root root      4096 Aug 30 13:25 sample_data


In [ ]:
# Mostrando os arquivos baixados
!ls -l

Os arquivos do ENEM são recebidos em formato compactado, para fazer a descompactação existem duas opções: usando comandos linux shell (através do unzip) ou usando o pacote zipfile do Python. Como este exercício é uma prática em Python e PySpark, usei o pacote zipfile. As recomendações para descompactação foram feitas pelo chatGPT.

In [ ]:
# Importando pacote para descompactar o arquivo ENEM 2020
import zipfile

In [ ]:
# Medindo o tempo de descompactação
%%time

with zipfile.ZipFile('microdados_enem_2020.zip', 'r') as zip_ref:
    zip_ref.extractall('enem_2020')

CPU times: user 13 s, sys: 5.55 s, total: 18.5 s
Wall time: 22.5 s


Outro passo necessário é organizar os arquivos da descompactação, usei apenas o arquivo principal para a transformação. Por isso ele será movido para a pasta /raw.

Mover arquivos entre pastas usando comandos linux shell é bem simples através do comando mv. Novamente, para manter o treinamento voltado para Python, usei o pacote shutil para realizar a movimentação e seus comandos foram passados pelo chatGPT.

In [ ]:
import shutil

In [ ]:
"""
O código comentado abaixo foi passado pelo chat GPT,
porém não é suficiente para realizar a movimentação da forma necessária.
A documentação foi consultada para conseguir os detalhes faltantes.
"""
# shutil.move('arquivo.txt', 'nova_pasta/')

# Atribuuindo os endereços dos dados a variáveis
source = "/content/enem_2020/DADOS/MICRODADOS_ENEM_2020.csv"
destination = "/content/enem_2020/raw/MICRODADOS_ENEM_2020.csv"

# Realizando a movimentação
shutil.move(source, destination)

'/content/enem_2020/raw/MICRODADOS_ENEM_2020.csv'

In [ ]:
import pyspark.sql.functions

dados_enem = spark.read.csv('/content/enem_2020/raw/MICRODADOS_ENEM_2020.csv', header= True, sep=';', encoding='ISO-8859-1')
dados_enem.show(5)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [ ]:
"""
Verificando as colunas do DataFrame, avaliando junto à documetação do dicionário dos dados
sobre o conteúdo, formato...
"""
dados_enem.columns

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

# Transformação dos dados

In [ ]:
"""
Para realizar o tratamento de forma mais eficiente, o arquivo será transformado em Parquet.
Segundo chat GPT, podemos atribuir o destino do arquivo em uma variável para usar na função
de salvar o arquivo no formato desejado.
"""

# Caminho para salvar o arquivo Parquet
parquet_path = "/content/enem_2020/refined/MICRODADOS_ENEM_2020.parquet"

# Salvar o DataFrame como arquivo Parquet
mida.write.mode("overwrite").parquet(parquet_path)

In [ ]:
%%time

# Pra ler o arquivo Parquet pode usar a mesma variável criada para alocar o aquivo na célula anterior
mida_parquet = spark.read.parquet(parquet_path)
mida_parquet.show(10)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

Parte do processo de ETL é a seleção das colunas que contém os dados importantes para análises posteriores pela equipe de Analytics.

Em um dataset que contém muitas colunas como esse, para manter o código limpo é interessante colocar as colunas selecionadas em uma variável antes de colocar na função. Isso é realizado utilizando o documento de dicionário de dados.

In [ ]:
# Imprimindo as colunas em uma lista
list(mida_parquet.columns)

['NU_INSCRICAO',
 'NU_ANO',
 'TP_FAIXA_ETARIA',
 'TP_SEXO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'CO_MUNICIPIO_PROVA',
 'NO_MUNICIPIO_PROVA',
 'CO_UF_PROVA',
 'SG_UF_PROVA',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'NU_NOTA_CN',
 'NU_NOTA_CH',
 'NU_NOTA_LC',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TP_LINGUA',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'TP_STATUS_REDACAO',
 'NU_NOTA_COMP1',
 'NU_NOTA_COMP2',
 'NU_NOTA_COMP3',
 'NU_NOTA_COMP4',
 'NU_NOTA_COMP5',
 'NU_NOTA_REDACAO',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009

In [ ]:
# Selecionando as colunas para a área de Analytics
dropcols = ('TX_RESPOSTAS_CN',
 'TX_RESPOSTAS_CH',
 'TX_RESPOSTAS_LC',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_CN',
 'TX_GABARITO_CH',
 'TX_GABARITO_LC',
 'TX_GABARITO_MT',
 'Q001',
 'Q002',
 'Q003',
 'Q004',
 'Q005',
 'Q006',
 'Q007',
 'Q008',
 'Q009',
 'Q010',
 'Q011',
 'Q012',
 'Q013',
 'Q014',
 'Q015',
 'Q016',
 'Q017',
 'Q018',
 'Q019',
 'Q020',
 'Q021',
 'Q022',
 'Q023',
 'Q024',
 'Q025')

In [ ]:
mida_parquet = mida_parquet.drop(*dropcols)
mida_parquet.show(10)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|CO_MUNICIPIO_ESC|   NO_MUNICIPIO_ESC|CO_UF_ESC|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|TP_SIT_FUNC_ESC|CO_MUNICIPIO_PROVA| NO_MUNICIPIO_PROVA|CO_UF_PROVA|SG_UF_PROVA|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|CO_PROV

In [ ]:
mida_parquet.count()

5783109